In [ ]:
import torch
import torch.nn as nn
from torch import optim
from torchvision import models
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
from torchvision.transforms import ToTensor
from torch.utils.data import Subset, random_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_score, recall_score, f1_score

In [ ]:
hushem = './backend/datasets/HuSHem/'
hushem_data_folder = datasets.ImageFolder(hushem)

In [ ]:
# Split the data into training and test sets
train_size = int(0.8 * len(hushem_data_folder))
test_size = len(hushem_data_folder) - train_size
train_data, test_data = random_split(hushem_data_folder, [train_size, test_size])

In [ ]:
# Transformation des images
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [ ]:
# # Charger les images avec ImageFolder
# hushem_data = datasets.ImageFolder(root=hushem, transform=transform)

# # Créer le dataloader
# dataloader = DataLoader(hushem_data, batch_size=32, shuffle=True)

# # Vérifier si les données sont correctement chargées
# for inputs, labels in dataloader:
#     print(inputs.shape, labels.shape)
#     break

In [ ]:
# Charger les images avec ImageFolder
hushem_data = datasets.ImageFolder(root=hushem, transform=transform)

# Séparer les données en ensembles d'entraînement et de test
train_size = int(0.8 * len(hushem_data))
test_size = len(hushem_data) - train_size
train_data, test_data = random_split(hushem_data, [train_size, test_size])

In [ ]:
# Créer les dataloaders
train_dataloader = DataLoader(train_data, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=32, shuffle=False)


In [ ]:
# Charger le modèle
model = models.resnet18(pretrained=True)
model.fc = nn.Linear(model.fc.in_features, len(hushem_data.classes))

In [ ]:
# Définir la fonction de perte
criterion = nn.CrossEntropyLoss()

In [ ]:
# Choisir l'optimiseur
optimizer = optim.SGD(model.parameters(), lr=0.001)


In [ ]:
# Entraîner le modèle
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for inputs, labels in train_dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

In [ ]:
# Évaluer le modèle
model.eval()
with torch.no_grad():
    y_true = []
    y_pred = []
    for inputs, labels in test_dataloader:
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        y_true.extend(labels.tolist())
        y_pred.extend(preds.tolist())

In [ ]:
# Calculer les métriques d'évaluation
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

In [ ]:
# Afficher les résultats
print(f'Precision: {precision:.4f}')
print(f'Recall: {recall:.4f}')
print(f'F1 Score: {f1:.4f}')